In [1]:
import math
import numpy as np

In [2]:
def parse_file(name):
    array = np.empty([1_000_000, 10], dtype=np.int)
    with open(name, 'r') as file:
        for idx, line in enumerate(file):
            array[idx] = np.array(list(map(int, line.strip('\n').split(' '))), dtype=np.int)
    return array

In [3]:
# Считываем файлы с данными в numpy массив
c1 = parse_file('./c1.txt')
c2 = parse_file('./c2.txt')

In [4]:
session_size = 100
n_samples = 1000
sample_size = 100
t_0 = 1.96

In [5]:
# Вычисляем стастику CTR (среднее количество сессий с хотя бы одним кликом)
CTR1, CTR2 = [], []
for _ in range(int(c1.shape[0] / session_size)):
    CTR1.append(np.count_nonzero(np.sum(c1[session_size * _ : session_size * (_ + 1)], axis=1) != 0) / session_size)
    CTR2.append(np.count_nonzero(np.sum(c2[session_size * _ : session_size * (_ + 1)], axis=1) != 0) / session_size)
CTR1 = np.array(CTR1)
CTR2 = np.array(CTR2)

In [6]:
# Вычисляем стастику CF (среднее количество сессий с кликом в первый результат)
CF1, CF2 = [], []
for _ in range(int(c1.shape[0] / session_size)):
    CF1.append(np.count_nonzero(c1[session_size * _ : session_size * (_ + 1), 0] != 0) / session_size)
    CF2.append(np.count_nonzero(c2[session_size * _ : session_size * (_ + 1), 0] != 0) / session_size)
CF1 = np.array(CF1)
CF2 = np.array(CF2)

In [7]:
# Функция для расчёта статистики ASL 
def calc_ASL(metric_1, metric_2):
    succ_cnt = 0
    for _ in range(n_samples):
        samples_idx = np.random.choice(range(len(metric_1)), size=sample_size, replace=True)

        mean_1, mean_2 = np.mean(metric_1[samples_idx]), np.mean(metric_2[samples_idx])
        var_1, var_2 = np.var(metric_1[samples_idx], ddof=1), np.var(metric_2[samples_idx], ddof=1)

        t = (mean_1 - mean_2) / math.sqrt(var_1 / n_samples + var_2 / n_samples)

        if t > t_0:
            succ_cnt += 1
    print('ASL: ', succ_cnt / n_samples)

In [8]:
calc_ASL(CTR1, CTR2)

ASL:  1.0


In [9]:
calc_ASL(CF1, CF2)

ASL:  1.0


Т.к. в обоих случаях значение ASL $\approx$ 1.0, следовательно гипотеза H0: CTR на двух сплитах можно
объяснить шумами - неверна.